# 添加新的量化方法

Transformers 支持并集成了许多量化方法，如 QLoRA、GPTQ、LLM.int8 和 AWQ。然而，还有一些其他量化方法尚未集成。为了更轻松地添加和使用这些量化方法，你应该使用 `HfQuantizer` 类。`HfQuantizer` 是一个内部辅助类，用于添加量化方法，而不是应用于每个 PyTorch 模块。

本指南将向你展示如何使用 `HfQuantizer` 类集成新的量化方法。

## 前提条件

在将新的量化方法集成到 Transformers 中之前，请确保你要添加的方法满足以下前提条件。目前，仅支持可以在 PyTorch 模块上运行的量化方法。

- 量化方法可以通过一个任何人都可以 pip 安装的 Python 包获得（如果你只能从源代码安装该包也是可以的）。理想情况下，pip 包中包含预编译的内核。
- 该方法可以在常用硬件（CPU、GPU 等）上运行。
- 该方法封装在一个 `nn.Module` 中（例如 `Linear8bitLt`、`Linear4bit`），并且量化线性层应具有以下定义：



这样，Transformers 模型可以通过将某些 `nn.Linear` 实例替换为目标类来轻松量化。

- 量化方法应该是可序列化的。你可以将量化后的权重保存到本地或推送到 Hub。
- 确保包含量化内核/原语的包是稳定的（没有频繁的破坏性更改）。

对于一些量化方法，它们可能需要通过数据校准（例如 AWQ）预先量化模型。在这种情况下，我们更倾向于仅在 Transformers 中支持推理，并让第三方库处理模型的量化过程。

## 构建新的 HfQuantizer 类

1. 在 [src/transformers/utils/quantization_config.py](https://github.com/huggingface/transformers/blob/abbffc4525566a48a9733639797c812301218b83/src/transformers/utils/quantization_config.py) 中创建一个新的量化配置类，并确保在 Transformers 主 `init` 文件中公开新的量化配置，即将其添加到 [src/transformers/**init**.py](https://github.com/huggingface/transformers/blob/abbffc4525566a48a9733639797c812301218b83/src/transformers/__init__.py) 的 `[_import_structure]` 对象中。
   
2. 在 [src/transformers/quantizers/](https://github.com/huggingface/transformers/tree/abbffc4525566a48a9733639797c812301218b83/src/transformers/quantizers) 目录下创建一个名为 `quantizer_your_method.py` 的新文件，并使其继承自 [src/transformers/quantizers/base.py::HfQuantizer](https://github.com/huggingface/transformers/blob/abbffc4525566a48a9733639797c812301218b83/src/transformers/quantizers/base.py#L28)。确保在 [src/transformers/quantizers/auto.py](https://github.com/huggingface/transformers/blob/abbffc4525566a48a9733639797c812301218b83/src/transformers/quantizers/auto.py) 中添加新的量化器和量化配置。

3. 为你的量化方法定义以下类属性/属性方法：
   
   - `requires_calibration`: 量化方法是否需要数据校准过程。如果设置为 `True`，则只能支持推理（带有量化权重）而不能支持推理和量化。
   - `required_packages`: 一个字符串列表，包含使用量化权重所需的包。你可能需要定义一些新的实用方法，例如 [transformers/src/utils/import_utils.py](https://github.com/huggingface/transformers/blob/abbffc4525566a48a9733639797c812301218b83/src/transformers/utils/import_utils.py) 中的 `is_auto_awq_available`。
   - `requires_parameters_quantization`: 仅在量化方法需要对底层 `nn.Parameter` 对象特别关注时才需要。例如，bitsandbytes 使用 `Params4bit` 和 `Int8Param`，这在量化模型时需要特别关注。大多数最近的量化方法将 int2/int4 权重打包到 `torch.uint8` 权重中，因此这个标志通常不需要（默认设置为 `False`）。
   - `is_serializable`: 一个属性方法，用于确定方法是否可序列化。
   - `is_trainable`: 一个属性方法，用于确定你是否可以在量化方法的基础上微调模型（无论是否使用 PEFT 方法）。

4. 编写 `validate_environment` 和 `update_torch_dtype` 方法。这些方法在创建量化模型之前被调用，以确保用户使用的配置正确。你可以参考其他量化器的实现方式。

5. 编写 `_process_model_before_weight_loading` 方法。在 Transformers 中，量化模型首先在 `"meta"` 设备上初始化，然后加载权重。这意味着 `_process_model_before_weight_loading` 方法负责操作模型骨架，以将某些模块（例如 `nn.Linear`）替换为目标模块（量化模块）。你可以在 [transformers/src/integrations/](https://github.com/huggingface/transformers/tree/abbffc4525566a48a9733639797c812301218b83/src/transformers/integrations) 目录下创建一个新文件，并在该目录的 `__init__.py` 文件中公开相关方法。最好的起点是查看其他量化方法，如 [quantizer_awq.py](https://github.com/huggingface/transformers/blob/abbffc4525566a48a9733639797c812301218b83/src/transformers/quantizers/quantizer_awq.py)。

6. 编写 `_process_model_after_weight_loading` 方法。此方法允许在加载权重后操作模型以实现额外功能。

7. 文档化一切！确保你的量化方法有文档记录，方法是在 `docs/source/en/quantization` 下添加一个新文件，并在 `docs/source/en/quantization/overview.md` 表格中添加一行。

8. 添加测试！首先在 `docker/transformers-quantization-latest-gpu` 中的夜间构建 Dockerfile 中添加包，然后在 `tests/quantization/xxx` 中添加一个新的测试文件。你可以参考其他量化方法的实现方式。